## Import stuff

In [25]:
import tensorflow as tf
from keras.losses import categorical_crossentropy, mean_squared_error
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.mixture import BayesianGaussianMixture, GaussianMixture

from tqdm import tqdm
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

print(tf.__version__)

def get_score(df_true, df_submission):
    for i in range(df_true.shape[1]):
        print("score for col ",i,": " , metrics.roc_auc_score(df_true[:,i], df_submission[:,i]))


2.8.0


## Load training data from csv


In [26]:
df = pd.read_csv("train_features_improved.csv")
labels_df = pd.read_csv("train_labels.csv")

X = np.array(df.iloc[:, 1:].values,  dtype=float)
Y = np.array(labels_df.iloc[:, 1:].values,  dtype=float)

#normalize
# Y[:,13] = (Y[:,13]-27)/73

X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=0)

print("X shape: ", X.shape)
print("Y shape: ", Y.shape)



X shape:  (18995, 546)
Y shape:  (18995, 15)


## The function that trains and rates models

In [27]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier, XGBRFClassifier


def find_best_model(col, X_train, Y_train, X_valid, Y_valid, X_test, X, Y):
    models = []

    Y_train = Y_train[:,col]
    Y_valid = Y_valid[:,col]
    # print(X_train.shape)
    # X_train = PolynomialFeatures(degree=2).fit_transform(X_train)
    # print(X_train.shape)
    # X_valid = PolynomialFeatures(degree=2).fit_transform(X_valid)
    # X_train = SelectKBest(chi2, k=35).fit_transform(X_train+0.01, Y_train)
    # X_valid = SelectKBest(chi2, k=35).fit_transform(X_valid+0.01, Y_valid)
    # X_train, Y_train = oversampler.fit_resample(X_train, Y_train)
    # X_train, Y_train = undersampler.fit_resample(X_train, Y_train)

    #task 1 and 2
    if(col<11):    
        print("training HistGradientBoostingClassifier for col ", col)
        model = HistGradientBoostingClassifier()
        model.fit(X_train, Y_train)
        models.append(model)
        
        print("training ExtraTrees for col ", col)
        model = ExtraTreesClassifier(n_estimators=100, random_state=0)
        model.fit(X_train, Y_train)
        models.append(model)

        print("training rfc for col ", col)
        rfc = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=10)
        rfc.fit(X_train, Y_train)
        models.append(rfc)
        
        print("training XGBRFClassifier 1 for col ", col)
        model = XGBRFClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, subsample=0.8, colsample_bytree=0.8, seed=10)
        model.fit(X_train, Y_train)
        models.append(model)
                
        print("training XGBRFClassifier 2 for col ", col)
        model = XGBRFClassifier()
        model.fit(X_train, Y_train)
        models.append(model)
                        
        print("training XGBClassifier 1 for col ", col)
        model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, subsample=0.8, colsample_bytree=0.8, seed=10)
        model.fit(X_train, Y_train)
        models.append(model)
                                
        print("training XGBClassifier 2 for col ", col)
        model = XGBClassifier()
        model.fit(X_train, Y_train)
        models.append(model)

        print("training catboost 1 for col ", col)
        model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, eval_metric='AUC', logging_level='Silent')
        model.fit(X_train, Y_train)
        models.append(model)
        
        print("training catboost 2 for col ", col)
        model = CatBoostClassifier(logging_level='Silent')
        model.fit(X_train, Y_train)
        models.append(model)
                
        print("training lgbm 1 for col ", col)
        model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, subsample=0.8, colsample_bytree=0.8, seed=10)
        model.fit(X_train, Y_train)
        models.append(model)
                        
        print("training lgbm 2 for col ", col)
        model = LGBMClassifier(boosting_type='dart')
        model.fit(X_train, Y_train)
        models.append(model)
                                
        print("training lgbm 3 for col ", col)
        model = LGBMClassifier(boosting_type='goss')
        model.fit(X_train, Y_train)
        models.append(model)
                                
        # print("training lgbm 4 for col ", col)
        # model = LGBMClassifier(boosting_type='rf')
        # model.fit(X_train, Y_train)
        # models.append(model)

        # print("training swish neural net for col ", col)
        # callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=3)
        # model = tf.keras.Sequential([
        #     layers.Dense(512, activation=tf.keras.activations.swish),
        #     # layers.BatchNormalization(),
        #     layers.Dense(512, activation=tf.keras.activations.swish),
        #     # layers.BatchNormalization(),
        #     layers.Dense(1, activation='sigmoid')
        # ])
        # model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001))
        # history = model.fit(X_train, Y_train, epochs=100, verbose=True,callbacks=[callback], validation_data=(X_valid, Y_valid))
        # print(len(history.history['loss']))
        # models.append(model)

    
    #task 3
    if(col>=11):
        # print("training improved neural net for col ", col)
        # callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=3, restore_best_weights=True)
        # model = tf.keras.Sequential([
        #     layers.Dense(512, activation=tf.keras.activations.relu),
        #     layers.Dense(512, activation=tf.keras.activations.relu),
        #     layers.Dense(1, activation='linear')
        # ])
        # model.compile(loss="mean_squared_error", optimizer=tf.keras.optimizers.Adam(0.001))
        # history = model.fit(X_train, Y_train, epochs=100, verbose=False,callbacks=[callback], validation_data=(X_valid, Y_valid))
        # print(len(history.history['loss']))
        # models.append(model)

        # print("training ridge for col ", col)
        # ridge = Ridge(alpha=1.0)
        # ridge.fit(X_train, Y_train)
        # models.append(ridge)

        print("training HistGradientBoostingRegressor for col ", col)
        model = HistGradientBoostingRegressor()
        model.fit(X_train, Y_train)
        models.append(model)




    
    print("done training all models for col ", col)

    best_score = 0
    best_model = None
    probabilistic_models = ["XGBClassifier","XGBRFClassifier","LGBMClassifier","BayesianGaussianMixture", "MLPClassifier", "SVC", "RandomForestClassifier", "HistGradientBoostingClassifier", "GradientBoostingClassifier", "ExtraTreesClassifier", "RandomForestRegressor", "DecisionTreeClassifier", "GaussianNB", "AdaBoostClassifier", "QuadraticDiscriminantAnalysis", "GaussianProcessClassifier","CatBoostClassifier"]
    for model in models:
        model_type = str(type(model)).split(".")[-1].split("'")[0]
        # print("testing model ", model_type)
        #if the model is probabilistic
        if model_type in probabilistic_models:
            pred = model.predict_proba(X_valid)[:,1]
        else:
            pred = model.predict(X_valid)
        if(col<11):  
            np.clip(pred, 0.0, 1.0, out=pred)
            # print(pred.min())
            # print(pred.max())
        # print("predictions shape: ", pred.shape)
        if(col>=11):
            score = 0.5 + 0.5 * np.maximum(0, metrics.r2_score(Y_valid, pred))
        else:
            score = metrics.roc_auc_score(Y_valid, pred)
        print("score for",type(model),": " , score)
        if score > best_score:
            best_score = score
            best_model = model

    print("best model: ", type(best_model),best_score)

    #doesn't seem to work
    # if str(type(best_model)).count("Sequential") > 0:
    #     best_model.fit(X,Y[:,col], epochs=15, verbose=False)
    # else:
    #     best_model.fit(X, Y[:,col])

    predictions = best_model.predict(X_test)
    # print("predictions shape: ", predictions.shape)

    return predictions,best_score

## Writing results

In [28]:
X_test = pd.read_csv("test_features_improved.csv")
X_test = np.array(X_test.iloc[:, 1:].values,  dtype=float)

data = []
scores = []

for i in range(0,15):
# for i in range(0,1):
# for i in range(10,11):
# for i in range(11,15):
# for i in range(13,14):
    # if i == 11:
    #     X = np.array(pd.read_csv("train_features_imputed.csv").iloc[:, 1:].values,  dtype=float)
    #     X_test = np.array(pd.read_csv("test_features_imputed.csv").iloc[:, 1:].values,  dtype=float)

    # pred = find_best_model(i, X, Y, X, Y, X_test,X,Y)
    pred,best_score = find_best_model(i, X[:int(X.shape[0]*0.8)], Y[:int(Y.shape[0]*0.8)], X[int(X.shape[0]*0.8):], Y[int(Y.shape[0]*0.8):], X_test,X,Y)
    # pred = find_best_model(i, X[int(X.shape[0]*0.2):], Y[int(Y.shape[0]*0.2):], X[:int(X.shape[0]*0.2)], Y[:int(Y.shape[0]*0.2)], X_test,X,Y)
    # print("pred shape: ", pred.shape)
    pred = pred.reshape(pred.shape[0],1)
    data.append(pred)
    scores.append(best_score)





training HistGradientBoostingClassifier for col  0
training ExtraTrees for col  0
training rfc for col  0
training catboost for col  0
training XGBRFClassifier 1 for col  0
training XGBRFClassifier 2 for col  0
training XGBClassifier 1 for col  0
training XGBClassifier 2 for col  0
training catboost 1 for col  0
training catboost 2 for col  0
training lgbm 1 for col  0
training lgbm 2 for col  0
training lgbm 3 for col  0
done training all models for col  0
score for <class 'sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingClassifier'> :  0.9304215660536388
score for <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> :  0.9265086772691473
score for <class 'sklearn.ensemble._forest.RandomForestClassifier'> :  0.9206444392245727
score for <class 'catboost.core.CatBoostClassifier'> :  0.9296571235129124
score for <class 'xgboost.sklearn.XGBRFClassifier'> :  0.9235241388363065
score for <class 'xgboost.sklearn.XGBRFClassifier'> :  0.9218002570019205
score

In [29]:
print(np.mean(scores[:10]))
print(np.mean(scores[10]))
print(np.mean(scores[11:]))

print(np.mean([np.mean(scores[:10]),np.mean(scores[10]),np.mean(scores[11:])]))


0.8266314839747395
0.6910255245708058
0.766735835720222
0.7614642814219225


In [30]:
np_array = np.array(data).transpose()[0]

#un-normalize
# np_array[:,13] = (np_array[:,13]*73)+27

col_names = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2', 'LABEL_Sepsis', 'LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
print(len(col_names))
new_df = pd.DataFrame(np_array, columns=col_names)

output_csv = pd.read_csv("sample.csv")
output_csv.update(new_df)
output_csv.to_csv('submit.zip', index=False, float_format='%.3f', compression='zip')
output_csv.to_csv('submit.csv', index=False, float_format='%.3f')


15
